<a href="https://colab.research.google.com/github/affaq-baig/Itinerary-recommendation/blob/main/itinary_recommendation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163478 sha256=9170140ee4c38b3038404c4dfb052c4b4b753907d1805b6270f023eb442c3cb1
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Load your dataset
df = pd.read_csv('/content/data6.csv')

# Display the first few rows of the dataset
print("Data Head:")
print(df.head())

# Load data into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'destination', 'rating']], reader)

# Build a user-based collaborative filtering model
model = KNNBasic(sim_options={'user_based': True})
trainset = data.build_full_trainset()
model.fit(trainset)

# Prompt new user for their preferences
print("Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):")
user_category_input = input()

# Assume a new user has a user_id of -1 (replace with an appropriate strategy)
new_user_id = -1

# Check if the new user's preferences exist in the dataset
existing_preferences = set(df[df['user_id'] == new_user_id]['destination'].unique()).intersection(set(df[df['category'] == user_category_input]['destination'].unique()))

# Add new ratings to the original dataset only for unseen destinations
new_ratings = [(new_user_id, destination, 0) for destination in df[df['category'] == user_category_input]['destination'].unique() if destination not in existing_preferences]

# Handle the case where all destinations are already rated
if not new_ratings:
    print("All destinations in the preferred category are already rated by the new user.")
else:
    # Add new ratings to the dataset
    df = pd.concat([df, pd.DataFrame(new_ratings, columns=['user_id', 'destination', 'rating'])], ignore_index=True)

    # Load updated data into Surprise format
    data = Dataset.load_from_df(df[['user_id', 'destination', 'rating']], reader)

    # Build a new user-based collaborative filtering model with the updated data
    trainset = data.build_full_trainset()
    model.fit(trainset)

    # Get recommendations for the new user
    unseen_destinations = df[df['user_id'] == new_user_id]['destination'].unique()
    predictions = [(destination, model.predict(new_user_id, destination).est) for destination in unseen_destinations]

    # Sort predictions by estimated rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)

    # Get top-N recommended destinations
    top_recommendations = sorted_predictions[:5]
    print(f"Top Recommendations for the new user in the category {user_category_input}:")
    for recommendation in top_recommendations:
        print(f"- {recommendation[0]} (Estimated Rating: {recommendation[1]:.2f})")


Data Head:
   user_id       destination  rating    category
0        1  Karimabad Bazaar     4.5      Market
1        1      Eagle's Nest     4.2      Hiking
2        1        Altit Fort     3.8  Historical
3        2    Hopper Glacier     4.0    Trekking
4        2         Ultar Sar     4.6    Mountain
Computing the msd similarity matrix...
Done computing similarity matrix.
Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):
Historical
Computing the msd similarity matrix...
Done computing similarity matrix.
Top Recommendations for the new user in the category Historical:
- Altit Fort (Estimated Rating: 1.00)
- Baltit Fort (Estimated Rating: 1.00)
- Pakistan Monument (Estimated Rating: 1.00)
- Khaplu Fort (Estimated Rating: 1.00)
- Shigar Fort (Estimated Rating: 1.00)


In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Load your dataset
df = pd.read_csv('/content/data6.csv')

# Display the first few rows of the dataset
print("Data Head:")
print(df.head())

# Load data into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'destination', 'rating']], reader)

# Build a user-based collaborative filtering model
model = KNNBasic(sim_options={'user_based': True})
trainset = data.build_full_trainset()
model.fit(trainset)

# Prompt user for their preferred category
print("Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):")
user_category_input = input()

# Prompt user for the number of days they want to travel
num_days = int(input("How many days do you want to travel? "))

# Filter destinations based on the user's preferred category
category_destinations = df[df['category'] == user_category_input]['destination'].unique()

# Check if there are destinations in the preferred category
if not category_destinations.any():
    print(f"No destinations found in the category: {user_category_input}")
else:
    # Get the mean rating of the preferred category
    category_mean_rating = df[df['category'] == user_category_input]['rating'].mean()

    # Get recommendations for the user in the preferred category
    unseen_destinations = [destination for destination in category_destinations if destination not in df['destination'].unique()]

    # Predict ratings for unseen destinations in the preferred category
    predictions = [(destination, model.predict(0, destination).est if destination in unseen_destinations else category_mean_rating) for destination in category_destinations]

    # Sort predictions by estimated rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)

    # Get top-N recommended destinations in the preferred category
    top_recommendations = sorted_predictions[:num_days]  # Adjust the number of recommendations based on the user's input
    print(f"\nTop {num_days} Recommendations in the category {user_category_input} for a {num_days}-day itinerary:")
    for recommendation in top_recommendations:
        print(f"- {recommendation[0]} (Estimated Rating: {recommendation[1]:.2f})")


Data Head:
   user_id       destination  rating    category
0        1  Karimabad Bazaar     4.5      Market
1        1      Eagle's Nest     4.2      Hiking
2        1        Altit Fort     3.8  Historical
3        2    Hopper Glacier     4.0    Trekking
4        2         Ultar Sar     4.6    Mountain
Computing the msd similarity matrix...
Done computing similarity matrix.
Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):
Mountain
How many days do you want to travel? 7

Top 7 Recommendations in the category Mountain for a 7-day itinerary:
- Ultar Sar (Estimated Rating: 4.34)
- Khunjerab Pass (Estimated Rating: 4.34)
- Spantik Peak (Estimated Rating: 4.34)
- Mustansar (Estimated Rating: 4.34)
- Karakoram Range (Estimated Rating: 4.34)
- Siafu Sar (Estimated Rating: 4.34)
- Trango Towers (Estimated Rating: 4.34)


In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Load your dataset
df = pd.read_csv('/content/data6.csv')

# Display the first few rows of the dataset
print("Data Head:")
print(df.head())

# Load data into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'destination', 'rating']], reader)

# Build a user-based collaborative filtering model
model = KNNBasic(sim_options={'user_based': True})
trainset = data.build_full_trainset()
model.fit(trainset)

# Prompt user for their preferred category
print("Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):")
user_category_input = input()

# Prompt user for the number of days they want to travel
num_days = int(input("How many days do you want to travel? "))

# Filter destinations based on the user's preferred category
category_destinations = df[df['category'] == user_category_input]['destination'].unique()

# Check if there are destinations in the preferred category
if not category_destinations.any():
    print(f"No destinations found in the category: {user_category_input}")
else:
    # Get the mean rating of the preferred category
    category_mean_rating = df[df['category'] == user_category_input]['rating'].mean()

    # Get recommendations for the user in the preferred category
    unseen_destinations = [destination for destination in category_destinations if destination not in df['destination'].unique()]

    # Predict ratings for unseen destinations in the preferred category
    predictions = [(destination, model.predict(0, destination).est if destination in unseen_destinations else category_mean_rating) for destination in category_destinations]

    # Sort predictions by estimated rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)

    # Get top-N recommended destinations in the preferred category
    top_recommendations = sorted_predictions[:num_days]  # Adjust the number of recommendations based on the user's input

    # Organize recommendations day by day
    itinerary = {}
    for day, recommendation in enumerate(top_recommendations, start=1):
        itinerary[f"Day {day}"] = {"Destination": recommendation[0], "Estimated Rating": recommendation[1]}

    print(f"\nItinerary for {num_days}-day trip in the category {user_category_input}:")
    for day, details in itinerary.items():
        print(f"{day}: {details['Destination']} (Estimated Rating: {details['Estimated Rating']:.2f})")


Data Head:
   user_id       destination  rating    category
0        1  Karimabad Bazaar     4.5      Market
1        1      Eagle's Nest     4.2      Hiking
2        1        Altit Fort     3.8  Historical
3        2    Hopper Glacier     4.0    Trekking
4        2         Ultar Sar     4.6    Mountain
Computing the msd similarity matrix...
Done computing similarity matrix.
Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):
Mountain
How many days do you want to travel? 7

Itinerary for 7-day trip in the category Mountain:
Day 1: Ultar Sar (Estimated Rating: 4.34)
Day 2: Khunjerab Pass (Estimated Rating: 4.34)
Day 3: Spantik Peak (Estimated Rating: 4.34)
Day 4: Mustansar (Estimated Rating: 4.34)
Day 5: Karakoram Range (Estimated Rating: 4.34)
Day 6: Siafu Sar (Estimated Rating: 4.34)
Day 7: Trango Towers (Estimated Rating: 4.34)


In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Load your dataset
df = pd.read_csv('/content/data6.csv')

# Display the first few rows of the dataset
print("Data Head:")
print(df.head())

# Load data into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'destination', 'rating']], reader)

# Build a user-based collaborative filtering model
model = KNNBasic(sim_options={'user_based': True})
trainset = data.build_full_trainset()
model.fit(trainset)

# Prompt user for their preferred category
print("Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):")
user_category_input = input()

# Prompt user for the number of days they want to travel
num_days = int(input("How many days do you want to travel? "))

# Filter destinations based on the user's preferred category
category_destinations = df[df['category'] == user_category_input]['destination'].unique()

# Check if there are destinations in the preferred category
if not category_destinations.any():
    print(f"No destinations found in the category: {user_category_input}")
else:
    # Get the mean rating of the preferred category
    category_mean_rating = df[df['category'] == user_category_input]['rating'].mean()

    # Get recommendations for the user in the preferred category
    unseen_destinations = [destination for destination in category_destinations if destination not in df['destination'].unique()]

    # Predict ratings for unseen destinations in the preferred category
    predictions = [(destination, model.predict(0, destination).est if destination in unseen_destinations else category_mean_rating) for destination in category_destinations]

    # Sort predictions by estimated rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)

    # Get top-N recommended destinations in the preferred category
    top_recommendations = sorted_predictions[:min(num_days * 4, len(sorted_predictions))]  # Adjust the number of recommendations based on the user's input

    # Organize recommendations day by day
    itinerary = {}
    for day, recommendation in enumerate(top_recommendations, start=1):
        if day <= num_days:
            itinerary[f"Day {day}"] = {"Destination": recommendation[0], "Estimated Rating": recommendation[1]}

    print(f"\nItinerary for {num_days}-day trip in the category {user_category_input}:")
    for day, details in itinerary.items():
        print(f"{day}: {details['Destination']} (Estimated Rating: {details['Estimated Rating']:.2f})")


Data Head:
   user_id       destination  rating    category
0        1  Karimabad Bazaar     4.5      Market
1        1      Eagle's Nest     4.2      Hiking
2        1        Altit Fort     3.8  Historical
3        2    Hopper Glacier     4.0    Trekking
4        2         Ultar Sar     4.6    Mountain
Computing the msd similarity matrix...
Done computing similarity matrix.
Please enter your preferred category of travel (e.g., Historical, Adventure, Scenic Spot):
Scenic Spot
How many days do you want to travel? 88

Itinerary for 88-day trip in the category Scenic Spot:
Day 1: Rakaposhi View Point (Estimated Rating: 4.29)
Day 2: Attabad Lake (Estimated Rating: 4.29)
Day 3: Passu Cones (Estimated Rating: 4.29)
Day 4: Hunza River (Estimated Rating: 4.29)
Day 5: Cherry Blossom Point (Estimated Rating: 4.29)
Day 6: Karakoram Highway (Estimated Rating: 4.29)
Day 7: Khunjerab Pass (Estimated Rating: 4.29)
Day 8: Deosai Plains (Estimated Rating: 4.29)
Day 9: Sheosar Lake (Estimated Rating: 4.